In [1]:
from shapely.geometry import Point 
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import datetime
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

# Adding csv data

In [2]:
tornado_911 = pd.read_csv('../data/911_Phone_Calls_Tornado_030320.csv')
cadd_tornado = pd.read_csv('../data/Computer_Aided_Dispatch_Data_Tornado_Incidents_030320.csv', skiprows = 3)
derecho_911 = pd.read_csv('../data/911_Phone_Calls_Derecho_050320_050420.csv')
cadd_derecho = pd.read_csv('../data/Computer_Aided_Dispatch_Data_Derecho_Incidents_050320-050420.csv', skiprows = 3)

# Fixing columns

In [3]:
tornado_911.columns = ['time', 'calltype', 'lat', 'lng', 'cell_tower_address']
derecho_911.columns = ['time', 'calltype', 'lat', 'lng', 'cell_tower_address']
cadd_tornado.columns = ['time', 'location', 'lat', 'lng', 'incident_type']
cadd_derecho.columns = ['time', 'location', 'lat', 'lng', 'incident_type']

In [4]:
tornado_911.time = pd.to_datetime(tornado_911.time)
derecho_911.time = pd.to_datetime(derecho_911.time)
cadd_tornado.time = pd.to_datetime(cadd_tornado.time)
cadd_derecho.time = pd.to_datetime(cadd_derecho.time)

In [5]:
tornado_911['geometry'] = tornado_911.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
derecho_911['geometry'] = derecho_911.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
cadd_tornado['geometry'] = cadd_tornado.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)
cadd_derecho['geometry'] = cadd_derecho.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)

In [6]:
tornado_911.head()

,time,calltype,lat,lng,cell_tower_address,geometry
0,2020-03-03 00:34:00,911 Calls,36.072708,-86.665779,620 Richards RD - N Sector,POINT (-86.665779 36.072708)
1,2020-03-03 00:36:00,911 Calls,36.159634,-86.841904,4507D KENTUCKY AV - SE,POINT (-86.841904 36.159634)
2,2020-03-03 00:36:00,911 Calls,36.264153,-86.708862,329 THELMA ST - S Sector,POINT (-86.708862 36.264153)
3,2020-03-03 00:36:00,911 Calls,NaN,NaN,526 FOUNDRY DR,POINT (nan nan)
4,2020-03-03 00:36:00,911 Calls,36.233467,-86.721077,5007B RUSKIN AVE - N,POINT (-86.72107699999999 36.233467)


In [7]:
derecho_911.head()

,time,calltype,lat,lng,cell_tower_address,geometry
0,2020-05-03 16:30:00,911 Calls,36.247802,-86.716847,WIRELESS CALLER,POINT (-86.716847 36.247802)
1,2020-05-03 16:30:00,911 Calls,36.143108,-86.800621,1161 21ST AV S - SE,POINT (-86.80062100000001 36.143108)
2,2020-05-03 16:30:00,911 Calls,36.100516,-87.056329,8013 C MCCRORY LN - N,POINT (-87.05632900000001 36.100516)
3,2020-05-03 16:30:00,911 Calls,36.130021,-86.927819,5758 RIVER RD - SW,POINT (-86.927819 36.130021)
4,2020-05-03 16:31:00,911 Calls,36.142380,-86.881882,3744B ANNEX AVE - SE,POINT (-86.881882 36.14238)


In [8]:
cadd_tornado.head()

,time,location,lat,lng,incident_type,geometry
0,2020-03-03 00:36:34,6205 COCKRILL BEND CIR,36.182002,-86.894052,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89405212 36.18200191)
1,2020-03-03 00:37:24,7337 COCKRILL BEND BLVD,36.175666,-86.894978,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.89497756999999 36.175666)
2,2020-03-03 00:38:17,110 TUNE AIRPORT DR,36.179299,-86.883727,STRUCTURE FIRE COMMERCIAL INDUSTRIAL,POINT (-86.8837274 36.17929946)
3,2020-03-03 00:39:40,555 MARRIOTT DR,36.151108,-86.689921,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.6899214 36.1511077)
4,2020-03-03 00:39:42,1414 COUNTY HOSPITAL RD,36.177926,-86.845934,GENERAL ALARM COMMERCIAL INDUSTRIAL BUILD,POINT (-86.84593411 36.17792604)


In [9]:
cadd_derecho.head()

,time,location,lat,lng,incident_type,geometry
0,2020-05-03 16:31:07,901 KENWICK CT W,36.046877,-86.962526,ELECTRICAL HAZARD,POINT (-86.96252641 36.04687715)
1,2020-05-03 16:32:18,8036 ARBOR DR,36.082491,-86.940680,ELECTRICAL HAZARD,POINT (-86.94067952 36.08249081)
2,2020-05-03 16:32:23,119 DUE WEST AV E,36.248029,-86.716875,NOT ALERT COPD,POINT (-86.71687484 36.24802948)
3,2020-05-03 16:33:04,630 GALLATIN PKE S,36.256491,-86.714817,ABDOMINAL PAIN,POINT (-86.71481656 36.25649117)
4,2020-05-03 16:33:52,615 W HILLWOOD DR,36.129690,-86.879834,ELECTRICAL HAZARD,POINT (-86.87983392 36.12968971)


# Adding geojson data

In [10]:
damage_points = gpd.read_file('../data/damage_points.geojson')
path_polygons = gpd.read_file('../data/path_polygons.geojson')
tornado_paths = gpd.read_file('../data/tornado_paths.geojson')

# Fixing columns

In [11]:
damage_points.stormdate = damage_points.stormdate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
damage_points.surveydate = damage_points.surveydate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.stormdate = tornado_paths.stormdate.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.starttime = tornado_paths.starttime.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))
tornado_paths.endtime = tornado_paths.endtime.apply(lambda x: datetime.datetime.fromtimestamp(x / 1e3))

In [12]:
damage_points.head()

,objectid,stormdate,surveydate,event_id,damage,damage_txt,dod_txt,efscale,damage_dir,windspeed,...,device_id,qc,dod,surveytype,globalid,edit_user,edit_time,comments,path_guid,geometry
0,2047717,2020-03-03 00:57:00,2020-03-04 09:28:00,Mount Juliet,2,One- or Two-Family Residences (FR12),All walls collapsed,EF3,N/A,155,...,None,Y,9,None,{FE82BBAC-ABF8-4562-81F6-A527829FBFFE},None,None,none,None,POINT (-86.51628 36.18344)
1,2047909,2020-03-03 01:51:00,2020-03-05 13:51:00,Cookeville,27,Trees: Hardwood (TH),Trees uprooted,EF0,N/A,80,...,None,Y,3,None,{2417B433-0C00-4D6C-A059-45E0CFDA2AF6},None,None,none,None,POINT (-85.60552 36.17308)
2,2075630,2020-03-03 01:53:00,2020-04-13 01:37:00,Cookeville,2,One- or Two-Family Residences (FR12),"Loss of roof covering material (<20%), gutters...",EF0,N/A,80,...,None,Y,2,None,{10AC67A3-D75D-4E6E-82B8-DD307E183FC0},None,None,None,None,POINT (-85.57574 36.17740)
3,2060941,2020-03-03 00:43:00,2020-03-26 12:09:00,,1,Small Barns or Farm Outbuildings (SBO),Total destruction of building,EF1,N/A,95,...,None,Y,8,None,{A0744CB4-5993-4056-A13C-CF75182F5147},None,None,None,None,POINT (-86.76228 36.17340)
4,2060984,2020-03-03 00:43:00,2020-03-26 16:00:00,,7,Masonry Apartment or Motel Building (MAM),Loss of roof covering (<20%),EF0,N/A,80,...,None,Y,2,None,{DA83D428-084A-4ACA-A39E-DEE84AA28463},None,None,None,None,POINT (-86.75692 36.17568)


In [13]:
tornado_paths.head()

,objectid,event_id,stormdate,starttime,endtime,startlat,startlon,endlat,endlon,length,...,propdamage,edit_user,edit_time,created_user,created_date,last_edited_user,last_edited_date,comments,Shape__Length,geometry
0,879879,Nashville,2020-03-03 00:32:00,2020-03-03 00:32:00,2020-03-03 01:35:00,36.1725,-86.9478,36.1532,-85.8860,60.1317,...,-99,None,None,DATians,1583461561000,dat_editor,1618703241000,"An historic, long-track, strong EF-3 tornado b...",1.071075,"LINESTRING (-86.94783 36.17249, -86.94461 36.1..."
1,879077,Hwy 69 Ballpark Tornado,2020-03-03 06:04:00,2020-03-03 06:04:00,2020-03-03 06:12:00,32.6846,-87.6226,32.6863,-87.5492,4.3400,...,-99,None,None,DATians,1583353305000,DATians,1583358165000,None,0.074159,"LINESTRING (-87.62263 32.68464, -87.62182 32.6..."
2,879081,Lawley Tornado,2020-03-03 06:28:00,2020-03-03 06:28:00,2020-03-03 06:38:00,32.8648,-87.0496,32.8646,-86.9432,6.2100,...,-99,None,None,DATians,1583357522000,DATians,1583359203000,Path,0.106695,"LINESTRING (-87.04956 32.86476, -87.04019 32.8..."
3,879877,Cookeville,2020-03-03 01:48:00,2020-03-03 01:48:00,2020-03-03 01:56:00,36.1715,-85.6628,36.1709,-85.5135,8.3900,...,100000,None,None,DATians,1583461208000,DATians,1590771008000,"An historic, violent EF-4 tornado began in wes...",0.149870,"LINESTRING (-85.66285 36.17151, -85.61103 36.1..."
4,879878,Goffton,2020-03-03 02:05:00,2020-03-03 02:05:00,2020-03-03 02:06:00,36.1075,-85.4461,36.1068,-85.4420,0.2300,...,-99,None,None,DATians,1583461360000,DATians,1590604073000,A very brief and weak EF-0 tornado touched dow...,0.004130,"LINESTRING (-85.44610 36.10746, -85.44203 36.1..."


In [14]:
nashville_tornado = tornado_paths.loc[tornado_paths.event_id == 'Nashville']
nashville_tornado

,objectid,event_id,stormdate,starttime,endtime,startlat,startlon,endlat,endlon,length,...,propdamage,edit_user,edit_time,created_user,created_date,last_edited_user,last_edited_date,comments,Shape__Length,geometry
0,879879,Nashville,2020-03-03 00:32:00,2020-03-03 00:32:00,2020-03-03 01:35:00,36.1725,-86.9478,36.1532,-85.886,60.1317,...,-99,None,None,DATians,1583461561000,dat_editor,1618703241000,"An historic, long-track, strong EF-3 tornado b...",1.071075,"LINESTRING (-86.94783 36.17249, -86.94461 36.1..."


# Mapping the Nashville Tornado

In [21]:
style = {'fillColor': '#ff0000', 'color': 'red'}
style2 = {'fillColor': '#ff0000', 'color': 'lightgreen'}
maptest = folium.Map(location = [36.1627, -86.7816], zoom_start = 10)

folium.GeoJson(path_polygons).add_to(maptest)

folium.GeoJson(nashville_tornado.geometry, style_function = lambda x:style).add_to(maptest)

    
maptest